## encadrer, tirer les partie interesants dans l'image et les enregister 

In [10]:
import cv2
import numpy as np
import os , sys            
workpath = sys.path[0]

In [34]:
"""
path_img = f'{workpath}\datasheet\mmClasses-DataBase-IMG\mmClasses-DataBase-IMG\data_64_equilibre'
L = os.listdir(path_img)
img = f'{path_img}\{L[500]}'
img = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
"""
path_img = f'{workpath}\qq_img_relle'
L = os.listdir(path_img)
img = f'{path_img}\{L[0]}'
img = cv2.imread(img,cv2.IMREAD_GRAYSCALE)

In [56]:
print(path_img)
print(type(path_img))

E:\INSA-cour\A4A\Projet_multi\work-dictory\qq_img_relle
<class 'str'>


In [35]:
BGR_img =  cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
HSV_img = cv2.cvtColor(BGR_img, cv2.COLOR_BGR2HSV)

In [63]:
cv2.namedWindow("HSV_Image",0)
cv2.resizeWindow("HSV_Image",300,300)
cv2.imshow("HSV_Image", HSV_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [64]:
print(HSV_img.shape)

(1365, 2048, 3)


In [65]:
print(HSV_img[600][2000])

[  0   0 208]


In [18]:
def get_mask_image(hsv_image, lower, upper):
    """
    获取二值化图片
     1）通过阀值获取二值化图像
    """
    mask_image = cv2.inRange(hsv_image, lower, upper)  # 获取二值化图片
    
    cv2.namedWindow("Mask Image",0)
    cv2.resizeWindow("Mask Image",300,300)
    cv2.imshow("Mask Image", mask_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print(mask_image.shape)
    return mask_image

In [77]:
lower = np.array([0, 0, 10])
upper = np.array([0, 0, 180])
mask_image = get_mask_image(HSV_img, lower, upper)  # 获取二值化图像 通过HSV的高低阈值，提取图像部分区域

(1365, 2048)


In [67]:
def get_morphology_image(mask_image):
    """
    对二值化图像进行形态学处理-消除噪音影响
      1) 先进行膨胀处理
         膨胀就是使用算法，来将图像的边缘扩大些。作用就是将目标的边缘或者是内部的坑填掉,去掉较小的孔洞
      2) 再进行腐蚀处理
         腐蚀：腐蚀会把物体的边界腐蚀掉，主要应用在去除白噪声，也可以断开连在一起的物体
    """

    kernel = np.ones((5, 5), np.uint8)
    dialationIamge = cv2.dilate(mask_image, kernel, iterations=1)  # 膨胀处理
    eroded_iamge = cv2.erode(dialationIamge, kernel, iterations=1)  # 腐蚀处理

    cv2.namedWindow("Eroded Image",0)
    cv2.resizeWindow("Eroded Image",300,300)
    cv2.imshow("Eroded Image", eroded_iamge)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
      # 显示形态学处理过后的图片
    return eroded_iamge


In [76]:
eroded_image = get_morphology_image(mask_image)

In [69]:
def get_contours(eroded_iamge):
    """
    获取光伏组件串图片的轮廓并在原始图像上绘制轮廓
       输入为形态学处理后的二值图像，黑色为背景，白色为目标
       cv2.RETR_EXTERNAL 只检测外轮廓
       cv2.CHAIN_APPROX_SIMPLE
    """
    # contours是轮廓本身，hierarchy 每条轮廓对应的属性（后一个轮廓、前一个轮廓、父轮廓、内嵌轮廓的索引编号）
    contours, hierarchy = cv2.findContours(eroded_iamge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # 获取组件区域轮廓

    return contours

In [74]:
def get_and_save_areas(contours, original_image):
    """
       在原始图片中切割并保存轮廓区域
    """
    min_area = 300  # 过滤那些小面积的噪音区域 
    index = 0  # 用来标识组件区域的数量
    for numcontours, contour in enumerate(contours):
        image_area = cv2.contourArea(contour)
        if image_area > min_area:
            x, y, w, h = cv2.boundingRect(contour)
            module_image = original_image[y:y + h, x: x + w]
            index = index + 1
            path_storage = path_img + '/' + str(x) + '_' + str(y) + '_' + str(w) + '_' + str(h) + '_' + str(image_area) + '_' + str(index) + '.jpg'
            print(path_storage)
            cv2.imwrite(path_storage, module_image)  # 文件名以坐标（X、Y）,图片的高度与宽度,图片面积,文件序号

    return index

In [75]:
index = get_and_save_areas(get_contours(eroded_image), img)

E:\INSA-cour\A4A\Projet_multi\work-dictory\qq_img_relle/2005_1344_43_21_537.0_1.jpg
E:\INSA-cour\A4A\Projet_multi\work-dictory\qq_img_relle/1866_1259_138_106_10460.5_2.jpg
E:\INSA-cour\A4A\Projet_multi\work-dictory\qq_img_relle/1587_1200_287_165_31167.5_3.jpg
E:\INSA-cour\A4A\Projet_multi\work-dictory\qq_img_relle/278_1200_77_59_1303.0_4.jpg
E:\INSA-cour\A4A\Projet_multi\work-dictory\qq_img_relle/92_1179_43_77_413.0_5.jpg
E:\INSA-cour\A4A\Projet_multi\work-dictory\qq_img_relle/141_1178_58_76_2151.0_6.jpg
E:\INSA-cour\A4A\Projet_multi\work-dictory\qq_img_relle/84_1125_624_240_42177.0_7.jpg
E:\INSA-cour\A4A\Projet_multi\work-dictory\qq_img_relle/1262_0_137_118_13148.0_8.jpg
E:\INSA-cour\A4A\Projet_multi\work-dictory\qq_img_relle/713_0_387_184_41822.0_9.jpg
E:\INSA-cour\A4A\Projet_multi\work-dictory\qq_img_relle/644_0_70_18_344.5_10.jpg
E:\INSA-cour\A4A\Projet_multi\work-dictory\qq_img_relle/0_0_2048_1365_1370702.5_11.jpg
